In [1]:
import os
import sys
import pickle

%load_ext autoreload
%autoreload 2

In [2]:
WIN = 'D:/'
MAC = '/Volumes/Transcend2/'

osp = MAC

In [9]:
MNET_PATH = osp+'git/molnet/code/'
sys.path.append(MNET_PATH)
import mnet_utilities

## Preprocess the mgf files

Load the fragmentation files.

In [10]:
from mnet_utilities import load_mgf
mgf_vl = mnet_utilities.load_mgf(osp+'pymz/mzmine/210120/mzmine_frag/vl/frag_pos_vl.mgf')
mgf_mal = mnet_utilities.load_mgf(osp+'pymz/mzmine/210120/mzmine_frag/mal/frag_pos_mal.mgf')
mgf_zik = mnet_utilities.load_mgf(osp+'pymz/mzmine/210120/mzmine_frag/zik/gnps2.mgf')

Loaded 100 spectra
Loaded 200 spectra
Loaded 300 spectra
Loaded 100 spectra
Loaded 200 spectra
Loaded 300 spectra
Loaded 100 spectra
Loaded 200 spectra
Loaded 300 spectra


Remove the noise from the fragmentation files.

In [11]:
from metab_utils import remove_small_peaks
remove_small_peaks(mgf_vl, 1)
remove_small_peaks(mgf_zik, 1)
remove_small_peaks(mgf_mal, 1)

Change the retention times in the loaded fragmentation files using the GP models.

In [12]:
with open(osp+'pymz/mzmine/malmodel.txt', 'rb') as file:
    mmal = pickle.load(file)
with open(osp+'pymz/mzmine/vlmodel.txt', 'rb') as file:
    mvl = pickle.load(file)

In [13]:
from metab_utils import change_rt_in_mgf
change_rt_in_mgf(mgf_vl, mvl)
change_rt_in_mgf(mgf_mal, mmal)

Save the dictionaries of spectrum back to mgf files.

In [14]:
zik_mgf_file = os.path.join(osp, 'pymz/mzmine/210120/mzmine_frag/preprocessedzik.mgf')
vl_mgf_file = os.path.join(osp, 'pymz/mzmine/210120/mzmine_frag/preprocessedvl.mgf')
mal_mgf_file = os.path.join(osp, 'pymz/mzmine/210120/mzmine_frag/preprocessedmal.mgf')

In [15]:
from metab_utils import writeMgfFile, makeClusters
writeMgfFile(makeClusters(mgf_zik), zik_mgf_file)
writeMgfFile(makeClusters(mgf_vl), vl_mgf_file)
writeMgfFile(makeClusters(mgf_mal), mal_mgf_file)

ImportError: cannot import name 'writeMgfFile' from 'metab_utils' (/Volumes/Transcend2/git/pymzm/AnaMariaNotebooks/metab_utils.py)

## Create/Load the JoinAligner object

Align the fragmentation files from each dataset with the already aligned peaksets.

In [16]:
mal_changed_rt_csv = osp+'pymz/mzmine/210120/mzmine_frag/mal_changed_rt/frag_pos_mal_quant.csv'
vl_changed_rt_csv = osp+'pymz/mzmine/210120/mzmine_frag/vl_changed_rt/frag_pos_vl_quant.csv'
zik_csv = osp+'pymz/mzmine/210120/mzmine_frag/zik/gnps2_quant.csv'

Initialize the JoinAligner object.

In [17]:
from alignment import JoinAligner
aligner_gp = JoinAligner(mz_tolerance_absolute = 0.01,mz_tolerance_ppm = 10,rt_tolerance = 0.5)

Add the previously aligned peakset.

In [18]:
aligner_gp.add_file(os.path.join(osp+'pymz/mzmine/221119/samples_alignment/pp_aligned.csv'), None)

Add the fragmentation file for each dataset.

In [19]:
aligner_gp.add_file(mal_changed_rt_csv, mgf_mal)
aligner_gp.add_file(vl_changed_rt_csv, mgf_vl)
aligner_gp.add_file(zik_csv, mgf_zik)

Save the JoinAligner object.

In [20]:
with open(os.path.join(osp,'pymz/mzmine/aligner_gp.txt'), 'wb') as handle:
    pickle.dump(aligner_gp, handle)   

Load the JoinAligner object.

In [21]:
with open(os.path.join(osp,'pymz/mzmine/aligner_gp.txt'), 'rb') as file:
    aligner_gp = pickle.load(file)   